In [12]:
import numpy as np  
from matplotlib import pyplot as plt  
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os

In [35]:
gyro_files = ['1449601597.m_proc_gyro.dat','1449601657.m_proc_gyro.dat','1449601717.m_proc_gyro.dat'
              ,'1449601777.m_proc_gyro.dat','1449601855.m_proc_gyro.dat','1449601918.m_proc_gyro.dat'
              ,'1449601975.m_proc_gyro.dat','1449602035.m_proc_gyro.dat','1449602095.m_proc_gyro.dat'
              ,'1449602155.m_proc_gyro.dat','1449602215.m_proc_gyro.dat','1449602275.m_proc_gyro.dat']
acce_files = ['1449601597.m_raw_acc.dat','1449601657.m_raw_acc.dat','1449601717.m_raw_acc.dat'
              ,'1449601777.m_raw_acc.dat','1449601855.m_raw_acc.dat','1449601918.m_raw_acc.dat'
              ,'1449601975.m_raw_acc.dat','1449602035.m_raw_acc.dat','1449602095.m_raw_acc.dat'
              ,'1449602155.m_raw_acc.dat','1449602215.m_raw_acc.dat','1449602275.m_raw_acc.dat']
path_gyro = './gyro/'
path_acce = './acce/'

In [5]:
MAX_LENGTH = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def getdata(files):
    data=np.loadtxt(files)  
    data_xyz = data[:,1:]
    input_norm = (data_xyz-np.mean(data_xyz,axis = 0))/np.std(data_xyz,axis = 0)
    input_norm = torch.FloatTensor([input_norm])
    input_norm = input_norm.view(800,1,3)
    return input_norm

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

       
        self.gru = nn.GRU(input_size, hidden_size)

    def forward(self, input, hidden):
        #print('input',input)
        
        output = input
       # print('hidden:',hidden)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(output_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = F.relu(input)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [40]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

     #   self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.output_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.output_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.output_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
      #  embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(input)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [6]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        input_vector = input_tensor[ei].resize_((1,1,3))
        encoder_output, encoder_hidden = encoder(
            input_vector, encoder_hidden)
        #print('input_tensor[ei]',input_tensor[ei].size())
       # print('input_tensor[ei].size',input_tensor[ei].size(1))
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.zeros(1,1,3)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True #if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            #target_vector = target_tensor[di].resize_((1,1,3))
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
#             print('decoder_output.size', decoder_output.size())
#             print('target_vector.size', target_vector.size())
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di].resize_((1,1,3))  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di].resize_((1,1,3)))
#             if decoder_input.item() == EOS_token:
#                 break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [41]:
# train with attention decoder
teacher_forcing_ratio = 0.5


def train_att(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        input_vector = input_tensor[ei].resize_((1,1,3))
        encoder_output, encoder_hidden = encoder(
            input_vector, encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.zeros(1,1,3)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True #if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di].resize_((1,1,3))  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [7]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [47]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
    #criterion = nn.NLLLoss()
    criterion = nn.L1Loss()
    for i in range(len(gyro_files)):
        input_norm = getdata(os.path.join(path_gyro, gyro_files[i]))
        target_norm = getdata(os.path.join(path_acce, acce_files[i]))
        for iter in range(n_iters):
            #training_pair = training_pairs[iter - 1]
    #         input_tensor = training_pair[0]
    #         target_tensor = training_pair[1]
            input_tensor = input_norm[iter*20:(iter+1)*20]

            target_tensor = target_norm[iter*20:(iter+1)*20]
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
    #             print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
    #                                          iter, iter / n_iters * 100, print_loss_avg))
                print('iter,loss',iter, iter / n_iters * 100,print_loss_avg)

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)

In [43]:
def trainIters_att(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
    criterion = nn.L1Loss()
    for i in range(len(gyro_files)):
        input_norm = getdata(os.path.join(path_gyro, gyro_files[i]))
        target_norm = getdata(os.path.join(path_acce, acce_files[i]))
        for iter in range(n_iters):
            input_tensor = input_norm[iter*20:(iter+1)*20]

            target_tensor = target_norm[iter*20:(iter+1)*20]

            loss = train_att(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
#                 print_loss_total = 0
#                 print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
#                                              iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)

In [10]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [48]:
hidden_size = 256
encoder1 = EncoderRNN(3, hidden_size).to(device)
#attn_decoder1 = AttnDecoderRNN(hidden_size, 3, dropout_p=0.1).to(device)
decoder1 = DecoderRNN(hidden_size, 3)
trainIters(encoder1, decoder1, 40, print_every=5)
#trainIters_att(encoder1, attn_decoder1, 40, print_every=5)

iter,loss 0 0.0 0.2626708793640137
iter,loss 5 12.5 1.7235310745239256
iter,loss 10 25.0 1.1751797866821287
iter,loss 15 37.5 1.132084312438965
iter,loss 20 50.0 1.089485969543457
iter,loss 25 62.5 0.9762002754211426
iter,loss 30 75.0 1.0981698036193848
iter,loss 35 87.5 1.1712859725952147
iter,loss 0 0.0 1.1969023323059083
iter,loss 5 12.5 1.6833734130859377
iter,loss 10 25.0 1.17857027053833
iter,loss 15 37.5 1.1395240974426268
iter,loss 20 50.0 1.124070224761963
iter,loss 25 62.5 1.1150327682495118
iter,loss 30 75.0 1.0808489608764649
iter,loss 35 87.5 1.1880531883239747
iter,loss 0 0.0 1.30997802734375
iter,loss 5 12.5 1.3285737991333009
iter,loss 10 25.0 1.2308040237426758
iter,loss 15 37.5 1.498577117919922
iter,loss 20 50.0 1.1507832908630369
iter,loss 25 62.5 1.064268341064453
iter,loss 30 75.0 1.060208511352539
iter,loss 35 87.5 1.0624062156677248
iter,loss 0 0.0 1.332229995727539
iter,loss 5 12.5 1.2830263328552245
iter,loss 10 25.0 1.2650066184997557
iter,loss 15 37.5 1.2788